---
title: "Population Based HyperParametric Training"
format:
  html:
    code-fold: false
jupyter: python3
---


# Importing the required libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras as kr

# Preparing the dataset

In [ ]:
mnist = kr.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

train_data = {'X':x_train, 'Y':y_train}
test_data = {'X':x_test, 'Y':y_test}

# Template for Model

In [ ]:
model = kr.models.Sequential([
  kr.layers.Input((28,28)),
  kr.layers.Flatten(),
  kr.layers.Dense(128, activation='relu'),
  kr.layers.Dropout(0.2),
  kr.layers.Dense(10)
])

# Creating The Optimizer


In [ ]:
class PopulationBasedOptimizer:

    def __init__(self, model, NumberOfAgents, CutOffMeasure, NumberOfLearningIterations, InitialHyperParameter):
        self.listOfAgents = [kr.models.clone_model(model) for _ in range(NumberOfAgents)]
        self.HyperParameters = InitialHyperParameter
        self.NumberOfAgents = NumberOfAgents
        self.NumberOfLearningIterations = NumberOfLearningIterations
        self.CutOffMeasure = CutOffMeasure
        self.Iterations = NumberOfLearningIterations
        for Agent, rate in zip(self.listOfAgents,self.HyperParameters):
            optimizer = kr.optimizers.Adam(learning_rate=rate)
            Agent.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    def PBTOptimise(self, train_data, test_data):
        for iter in range(self.NumberOfLearningIterations):
            for model in self.listOfAgents:
                model.fit(train_data.get('X'), train_data.get('Y'), epochs=1, batch_size=10)
            current_loss = np.zeros_like(self.NumberOfAgents)
            for i,model in enumerate(self.listOfAgents):
                current_loss[i], accuracy = model.evaluate(test_data.get('X'), test_data.get('Y'), verbose=1)
                BadIndices = np.argsort(current_loss)[int(self.NumberOfAgents * self.CutOffMeasure):]
            Best = np.argmin(current_loss)
            for model in self.listOfAgents[BadIndices]:
                Best_weights = self.listOfAgents[Best].get_weights()
                model.set_weights(self.listOfAgents[Best].get_weights() + np.random.normal(0,1, shape(Best_weights)))
                optimizer = kr.optimizers.Adam(learning_rate=self.InitialHyperParameter[Best])
                model.compile(optimizer=optimizer, loss='categoricalcross_entropy', metrics=['accuracy'])

        for model in self.listOfAgents:
            model.fit(train_data.get('X'), train_data.get('Y'), epochs=1)
        current_loss = np.zeros_like(NumberOfAgents)
        for i,model in enumerate(self.listOfAgents):
            current_loss[i], accuracy = model.evaluate(test_data.get('X'), test_data.get('Y'), verbose=1)
            BadIndices = np.argsort(current_loss)[int(self.NumberOfAgents * self.CutOffMeasure):]
        Best = np.argmin(current_loss)
        # Please help
        return self.listOfAgents[Best]

In [ ]:
InitialHyperParameter = np.random.normal(0,1,10)
PBT = PopulationBasedOptimizer(model=model, NumberOfAgents=10, CutOffMeasure=0.2, NumberOfLearningIterations=20, InitialHyperParameter=InitialHyperParameter)
best_model = PBT.PBTOptimise(train_data = train_data, test_data = test_data)